<a href="https://colab.research.google.com/github/shkabh/PUS2024_SKaur/blob/main/12_10_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import layers

In [2]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [3]:
x_train.shape

(3601, 500)

In [4]:
np.unique(y_train)

array([-1,  1])

In [5]:
x_train = x_train[:, :, None]
x_test = x_test[:, :, None]

In [6]:
x_train.shape, x_test.shape

((3601, 500, 1), (1320, 500, 1))

In [7]:
n_classes = len(np.unique(y_train))
n_classes

2

In [8]:
idx = np.random.permutation(x_train.shape[0])
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [9]:
!pip install keras-nlp

In [10]:
def transformer_encoder(inputs):
  x = layers.LayerNormalization(epsilon = 1e-6)(inputs)
  x = layers.MultiHeadAttention(key_dim = 8, num_heads = 8, dropout = 0)(x, x)
  res = x + inputs

  x = layers.LayerNormalization(epsilon = 1e-6)(res)
  x = layers.Conv1D(filters = 2000, kernel_size = 1, activation = "relu")(x)
  x = layers.Conv1D(filters = inputs.shape[-1], kernel_size = 1)(x)

  return res + x

def build_model(input_shape, n_classes):
  inputs = keras.Input(shape = input_shape)
  x = inputs

  x = transformer_encoder(x)
  x = transformer_encoder(x)

  x = layers.GlobalMaxPooling1D(data_format = "channels_last")(x)

  x = layers.Dense(256, activation = "relu")(x)
  x = layers.Dense(50, activation = "relu")(x)

  outputs = layers.Dense(n_classes, activation = "softmax")(x)

  model = keras.Model(inputs, outputs)

  return model


In [11]:
input_shape = x_train.shape[1:]
model = build_model(input_shape, n_classes)

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 500, 1)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 500, 1)         │              2 │ input_layer[0][0]      │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 500, 1)         │            449 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 500, 1)         │              0 │ multi_head_attention[… │
│                           │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 500, 1)         │              2 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 500, 2000)      │          4,000 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 500, 1)         │          2,001 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 500, 1)         │              0 │ add[0][0],             │
│                           │                        │                │ conv1d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 500, 1)         │              2 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 500, 1)         │            449 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 500, 1)         │              0 │ multi_head_attention_… │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_3     │ (None, 500, 1)         │              2 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 500, 2000)      │          4,000 │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 500, 1)         │          2,001 │ conv1d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 500, 1)         │              0 │ add_2[0][0],           │
│                      

 Total params: 26,372 (103.02 KB)

 Trainable params: 26,372 (103.02 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 1e-4),
    metrics = ["sparse_categorical_accuracy"],
)

In [14]:
callbacks = [
    keras.callbacks.EarlyStopping(patience = 10, restore_best_weights= True),
]

In [15]:
model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    epochs = 100,
    batch_size = 12,
    verbose = 2,
)

Epoch 1/100
240/240 - 24s - 101ms/step - loss: 0.6951 - sparse_categorical_accuracy: 0.4962 - val_loss: 0.6916 - val_sparse_categorical_accuracy: 0.5326
Epoch 2/100
240/240 - 6s - 27ms/step - loss: 0.6936 - sparse_categorical_accuracy: 0.5045 - val_loss: 0.6952 - val_sparse_categorical_accuracy: 0.4674
Epoch 3/100
240/240 - 10s - 42ms/step - loss: 0.6927 - sparse_categorical_accuracy: 0.5191 - val_loss: 0.6915 - val_sparse_categorical_accuracy: 0.5326
Epoch 4/100
240/240 - 10s - 43ms/step - loss: 0.6941 - sparse_categorical_accuracy: 0.4934 - val_loss: 0.6920 - val_sparse_categorical_accuracy: 0.5284
Epoch 5/100
240/240 - 10s - 43ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.5049 - val_loss: 0.6922 - val_sparse_categorical_accuracy: 0.5381
Epoch 6/100
240/240 - 10s - 43ms/step - loss: 0.6940 - sparse_categorical_accuracy: 0.4958 - val_loss: 0.6940 - val_sparse_categorical_accuracy: 0.4674
Epoch 7/100
240/240 - 10s - 42ms/step - loss: 0.6935 - sparse_categorical_accuracy: 0.49